In [ ]:
import cv2
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from typing import Tuple, List, Optional
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class ImageProcessor:
    """Handles image preprocessing and augmentation"""
    
    def __init__(self, target_size: Tuple[int, int] = (224, 224)):
        self.target_size = target_size
        
    def preprocess_image(self, image_path: str) -> np.ndarray:
        """
        Preprocess image for model prediction
        
        Args:
            image_path: Path to the image file
            
        Returns:
            Preprocessed image array
        """
        try:
            # Load image using PIL
            img = Image.open(image_path)
            
            # Convert to RGB if needed
            if img.mode != 'RGB':
                img = img.convert('RGB')
            
            # Resize image
            img = img.resize(self.target_size, Image.Resampling.LANCZOS)
            
            # Convert to numpy array
            img_array = np.array(img)
            
            # Normalize pixel values to [0, 1]
            img_array = img_array.astype(np.float32) / 255.0
            
            # Add batch dimension
            img_array = np.expand_dims(img_array, axis=0)
            
            return img_array
            
        except Exception as e:
            logger.error(f"Error preprocessing image: {str(e)}")
            raise
    
    def enhance_image(self, image_path: str) -> str:
        """
        Enhance image quality using OpenCV
        
        Args:
            image_path: Path to the input image
            
        Returns:
            Path to the enhanced image
        """
        try:
            # Read image
            img = cv2.imread(image_path)
            
            # Apply enhancements
            # 1. Noise reduction
            img = cv2.bilateralFilter(img, 9, 75, 75)
            
            # 2. Histogram equalization for better contrast
            img_yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
            img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])
            img = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)
            
            # 3. Sharpening
            kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
            img = cv2.filter2D(img, -1, kernel)
            
            # Save enhanced image
            enhanced_path = image_path.replace('.', '_enhanced.')
            cv2.imwrite(enhanced_path, img)
            
            return enhanced_path
            
        except Exception as e:
            logger.error(f"Error enhancing image: {str(e)}")
            return image_path
    
    def extract_features(self, image_path: str) -> dict:
        """
        Extract basic image features
        
        Args:
            image_path: Path to the image
            
        Returns:
            Dictionary of image features
        """
        try:
            img = cv2.imread(image_path)
            
            # Basic properties
            height, width, channels = img.shape
            
            # Color analysis
            mean_color = np.mean(img, axis=(0, 1))
            
            # Brightness
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            brightness = np.mean(gray)
            
            # Contrast
            contrast = np.std(gray)
            
            return {
                'width': width,
                'height': height,
                'channels': channels,
                'mean_color_bgr': mean_color.tolist(),
                'brightness': float(brightness),
                'contrast': float(contrast)
            }
            
        except Exception as e:
            logger.error(f"Error extracting features: {str(e)}")
            return {}
